In [327]:
import geopandas as gpd
import pandas as pd
pd.set_option('display.max_columns', None)
import shapely
import json
from shapely.geometry import Point

In [302]:
LIST = gpd.read_parquet("../data/large_data/LIST_v0-4.parquet")

In [303]:
LIST.columns

Index(['LIST-ID', 'EDCS-ID', 'EDH-ID', 'trismegistos_uri', 'pleiades_id',
       'transcription', 'inscription', 'clean_text_conservative',
       'clean_text_interpretive_sentence', 'clean_text_interpretive_word',
       'clean_text_interpretive_word_EDCS', 'diplomatic_text', 'province',
       'place', 'inscr_type', 'status_notation', 'inscr_process', 'status',
       'partner_link', 'last_update', 'letter_size', 'type_of_inscription',
       'work_status', 'year_of_find', 'present_location', 'text_edition',
       'support_objecttype', 'support_material', 'support_decoration',
       'keywords_term', 'people', 'type_of_inscription_clean',
       'type_of_inscription_certainty', 'height_cm', 'width_cm', 'depth_cm',
       'material_clean', 'type_of_monument_clean',
       'type_of_monument_certainty', 'province_label_clean',
       'province_label_certainty', 'country_clean', 'country_certainty',
       'findspot_ancient_clean', 'findspot_ancient_certainty',
       'modern_region_cle

In [304]:
json_string = "[{'persname': {'name': [{'@type': 'praenomen', '#text': 'C.'}, {'@type': 'nomen', '#text': 'Iulius'}, {'@type': 'cognomen', '#text': 'Capito*'}]}, '@sex': 'M', '@{http://www.w3.org/XML/1998/namespace}id': 'HD057920_1'}, {'persname': {'name': [{'@type': 'praenomen', '#text': 'C.'}, {'@type': 'nomen', '#text': 'Iulius'}, {'@type': 'cognomen', '#text': 'Maximus+'}]}, '@sex': 'M', '@{http://www.w3.org/XML/1998/namespace}id': 'HD057920_2'}]"

In [305]:
json_string = json_string.replace("'", "\"")
json_string

'[{"persname": {"name": [{"@type": "praenomen", "#text": "C."}, {"@type": "nomen", "#text": "Iulius"}, {"@type": "cognomen", "#text": "Capito*"}]}, "@sex": "M", "@{http://www.w3.org/XML/1998/namespace}id": "HD057920_1"}, {"persname": {"name": [{"@type": "praenomen", "#text": "C."}, {"@type": "nomen", "#text": "Iulius"}, {"@type": "cognomen", "#text": "Maximus+"}]}, "@sex": "M", "@{http://www.w3.org/XML/1998/namespace}id": "HD057920_2"}]'

In [306]:
eval(json_string)

[{'persname': {'name': [{'@type': 'praenomen', '#text': 'C.'},
    {'@type': 'nomen', '#text': 'Iulius'},
    {'@type': 'cognomen', '#text': 'Capito*'}]},
  '@sex': 'M',
  '@{http://www.w3.org/XML/1998/namespace}id': 'HD057920_1'},
 {'persname': {'name': [{'@type': 'praenomen', '#text': 'C.'},
    {'@type': 'nomen', '#text': 'Iulius'},
    {'@type': 'cognomen', '#text': 'Maximus+'}]},
  '@sex': 'M',
  '@{http://www.w3.org/XML/1998/namespace}id': 'HD057920_2'}]

In [352]:
def eval_string_json(val):
    val = str(val).replace("'", "\"")
    try:
        val = json.loads(val)
    except:
        val = []
    return val
people_eval = LIST["people"].apply(eval_string_json)

In [357]:
people_eval = people_eval.apply(lambda x: json.dumps(x, ensure_ascii=False))

In [358]:
set([type(el) for el in people_eval])

{str}

In [359]:
LIST["people"] = people_eval

In [360]:
len(LIST)

525870

In [361]:
LIST.geometry.is_valid.sum()

525870

In [ ]:
from shapely.geometry import Point

In [330]:
def from_nones_to_emptypoints(geom):
    if geom is None:
        geom = Point()
    if isinstance(geom, shapely.geometry.collection.GeometryCollection):
        geom = Point()
    return geom
geometries = LIST.geometry.apply(from_nones_to_emptypoints)

In [335]:
LIST.geometry = geometries

In [338]:
len(LIST)

525870

In [345]:
LIST.geometry.is_valid.sum()

525870

In [287]:
# LIST = LIST[LIST.geometry.is_valid]

In [362]:
LIST.sample(20000, random_state=2).to_parquet("../data/large_data/LIST_sample.parquet", compression='snappy')

In [363]:
LIST.to_parquet("../data/large_data/LIST_v0-5.parquet")

In [364]:
LIST.sample(1000).to_file("../data/large_data/LIST_test.geojson", driver="GeoJSON")

In [ ]:
LIST.to_file("../data/large_data/LIST_v0-5.geojson", driver="GeoJSON")